In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deep-learning-for-msc-202324/seqs_train.csv
/kaggle/input/deep-learning-for-msc-202324/sample.csv
/kaggle/input/deep-learning-for-msc-202324/seqs_test.csv
/kaggle/input/deep-learning-for-msc-202324/labels_train.csv
/kaggle/input/deep-learning-for-msc-202324/test/1OGQ_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1NHI_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1R1H_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1VL5_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1JLI_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1JAD_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1R73_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1WU9_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1PAQ_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1XPP_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/test/1EZJ_1_A_test.csv
/kaggle/input/deep-learning-for-msc-202324/te

In [52]:
!pip install ax-platform

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



1. **Prepare liabraries that I may use in this coursework, if necessary,I will install extra library.**

In [53]:
import copy
import glob

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

from torch.optim import Adam
import torch.optim as optim

import os

2. **Load the realted data from kaggle**

In [54]:
pssm_train="/kaggle/input/deep-learning-for-msc-202324/train"
pssm_test="/kaggle/input/deep-learning-for-msc-202324/test"
seqs_train = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/seqs_train.csv")
seqs_test = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/seqs_test.csv")
labels_train = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/labels_train.csv")
samples = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/sample.csv")

3. **Preprocessing and train data**

In [55]:
"""merging seqs_train and labels_train DataFrames on the column 'PDB_ID'.  
ensuring that each sequence has its associated label. The result is stored back in seqs_train,
I enhanced with a label column named 'LABEL'."""

# Merge the dataframes->seqs_train and labels_train on the column 'PDB_ID'.
seqs_train = pd.merge(seqs_train, labels_train, on='PDB_ID')

# Setting up file path,match and training pssm files
path = '/kaggle/input/deep-learning-for-msc-202324/train'
filepaths = glob.glob(os.path.join(path, '*.csv'))

# Define a function to read processed PSSM data 
# Removes columns 'RES_NUM' and 'AMINO_ACID' to keep only the scoring matrix, 
# Converts this matrix into a NumPy array
def get_pssm_data(filepath):
    pssm_df = pd.read_csv(filepath)
    pssm_df.drop(['RES_NUM', 'AMINO_ACID'], axis=1, inplace=True)
    pssm_np = pssm_df.to_numpy()
        
    return pssm_np

# Adding PSSM Data to Training DataFrame

# Initialized all its values to None. 
seqs_train['PSSM'] = None

# Iterate over each row in seqs_train, 
for index, row in seqs_train.iterrows():
    pdb_id = row['PDB_ID']    
    # Constructing the file path for the corresponding PSSM file based on the 'PDB_ID' column and the predefined folder structure. 
    filepath = os.path.join(pssm_train, f'{pdb_id}_train.csv')
    
    # For each sequence, you read its PSSM data using get_pssm_data and store this NumPy array in the 'PSSM' column of seqs_train
    pssm_np = get_pssm_data(filepath)
    seqs_train.at[index, 'PSSM'] = pssm_np

# Print the train data to check
print(seqs_train.head())

     PDB_ID                                           SEQUENCE  \
0  1VBK_1_A  MNVVIVRYGEIGTKSRQTRSWFEKILMNNIREALVTEEVPYKEIFS...   
1  1QGV_1_A  MSYMLPHLHNGWQVDQAILSEEDRVVVIRFGHDWDPTCMKMDEVLY...   
2  1KX6_1_A                      HSQGTFTSDYSKYLDSRRAQDFVQWLMNT   
3  1V88_1_A  GSSGSSGIVMADWLKIRGTLKSWTKLWCVLKPGVLLIYKTQKNGQW...   
4  1ZY8_2_K  GDPIKILMPSLSPTMEEGNIVKWLKKEGEAVSAGDALCEIETDKAV...   

                                          SEC_STRUCT  \
0  CCEEEEEECCCCCCCCCCHHHHHHHHHHHHHHHHHHCCCCCCEEEE...   
1  CCCCCCECCCHHHHHHHHHCCCCCEEEEEEECCCCHHHHHHHHHHH...   
2                      CCCCCCCCCCCCCCCCCCCCCCHHHHCCC   
3  CCCCCCCCCEEEEEEECCCCCCCEEEEEEEECCEEEEECCCCCCCC...   
4  CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...   

                                                PSSM  
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...  
3  [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0

In [56]:
# Encoding Amino Acid Sequences
# Design a function for One-hot Encoding of Amino Acids
def encode_amino_acid_sequence(amino_acid_sequence):
    # Translate each amino acid into a one-hot encoded vector, map 20 standard amino acids to unique integers
    amino_acid_to_int = {
        'A': 0, 'C': 1, 'D': 2, 'E': 3, 
        'F': 4, 'G': 5, 'H': 6, 'I': 7, 
        'K': 8, 'L': 9, 'M': 10, 'N': 11, 
        'P': 12, 'Q': 13, 'R': 14, 'S': 15, 
        'T': 16, 'V': 17, 'W': 18, 'Y': 19
    }
    
    # The position corresponding to the amino acid's integer representation is set to 1, and all others are 0.
    one_hot_sequence = np.zeros((len(amino_acid_sequence), 20))
    for i, amino_acid in enumerate(amino_acid_sequence):
        if amino_acid in amino_acid_to_int:
            one_hot_sequence[i, amino_acid_to_int[amino_acid]] = 1
        else:
            pass
    
    return one_hot_sequence

# Padding Sequences for Uniform Length

# Involve padding these sequences to ensure they all have the same length
# Maximum sequence length is determined from the training data
max_length = max(seqs_train['SEQUENCE'].apply(len))
# Define a function that takes as input the amino acid sequences from the training dataset
# Apply one-hot encoding to each sequence, converting the amino acids into a binary format that's suitable for machine learning models.
def encode_sequences(sequences, max_length):
    numerical_sequences = [encode_amino_acid_sequence(seq) for seq in sequences]
    return numerical_sequences


# Define one structural labels
label_dict = {
    'H': 0, 'E': 1, 'C': 2
}

# Match the lengths of the corresponding padded amino acid sequences,
# Let each amino acid position own a corresponding structural label.
def encode_labels(labels, max_length):
    numerical_labels = [[label_dict[label] for label in seq] for seq in labels]
    padded_labels = [seq + [0] * (max_length - len(seq)) for seq in numerical_labels]
    return padded_labels

# Apply padding to both the sequences and their corresponding labels as part of the fifth step. 
# Begin by encoding the amino acid sequences and labels from the training set to ensure uniformity.
padded_sequences = encode_sequences(seqs_train['SEQUENCE'], max_length)
padded_labels = encode_labels(seqs_train['SEC_STRUCT'], max_length)


# Set and preparing PSSM Data
# Convert the data type of PSSM column to np.array too slow!!
# The PSSM data for each sequence is padded to ensure uniformity in length, aligning with the padded amino acid sequences and labels.
def encode_pssm(pssm):
    max_length = max([pssm.shape[0] for pssm in pssm])
    # padding to 0
    padded_pssm = [np.pad(pssm, ((0, max_length - pssm.shape[0]), (0, 0)), constant_values=0) for pssm in pssm]
    return padded_pssm

#padding the pssm column
padded_pssm = encode_pssm(seqs_train['PSSM'])
# Concatenate the list of arrays into a single NumPy array
pssm_array = np.stack(padded_pssm)
# Convert the entire array to a PyTorch tensor
pssm_tensor = torch.tensor(pssm_array, dtype=torch.float32)
label_tensor = torch.tensor(padded_labels, dtype=torch.long)

# Transpoese tensor to the correct one
pssm_tensor = pssm_tensor.permute(0, 2, 1)

# Print reshaped tensor
print(pssm_tensor.shape)
print(label_tensor.shape)

torch.Size([7150, 20, 1733])
torch.Size([7150, 1733])


**4. Split preocessed dataset to train set and validation set **

In [57]:
# 0.8 train + 0.2 val -> After testing and comparsion this split method is best
# 0.7 train + 0.3 val    
# 0.6 train + 0.4 val

# Define Proteindataset:
class ProteinDataset(Dataset):
    # Initialize a custom dataset for protein sequences, handling PSSM data with optional labels for supervised learning tasks.
    def __init__(self, pssm, labels=None):
        self.pssm = pssm
        self.labels = labels
        self.include_labels = labels is not None 
        
    def __len__(self):
        return len(self.pssm)
    
    def __getitem__(self, idx):
        sequence = self.pssm[idx]
        
        if self.include_labels:
            label = self.labels[idx]
            return sequence, label
        else:
            return sequence

        
# Assign corresponding object
protein_dataset = ProteinDataset(pssm_tensor, label_tensor)

train_size = int(0.8 * len(protein_dataset))
# train_size = int(0.7 * len(protein_dataset))
# train_size = int(0.6 * len(protein_dataset))
val_size = len(protein_dataset) - train_size
train_dataset, val_dataset = random_split(protein_dataset, [train_size, val_size])
# Design one DataLoader object for each set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [58]:
# Define a protein module
class ProteinStructurePredictor(nn.Module):
    def __init__(self):
        super(ProteinStructurePredictor, self).__init__()
        
        # Define the convolutional layers
        # Update your model's first layer to accept the new input size
        self.conv1 = nn.Conv1d(in_channels=20, out_channels=64, kernel_size=3, padding=1)  # 20 amino acids
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(in_channels=256, out_channels=3, kernel_size=3, padding=1)  # out_channels = 3 ->H, E, C
        
        # Optional: Batch normalization layers after each Conv layer
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        
        # The output layer is already included as conv4

    def forward(self, x):
        # Assuming x is of shape (batch_size, 20, seq_len), where 20 amino acids
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)  
        return x


In [59]:
# Create a model object and put it on the GPU for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ProteinStructurePredictor().to(device)
#Define a loss function and an optimizer
#The loss function is the cross entropy loss function

criterion = nn.CrossEntropyLoss()
#The optimizer is the Adam optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# train the model
epochs = 51
for epoch in range(epochs):
    model.train()
    train_loss = 0
    # correct = 0
    # total = 0
    for pssm, label in train_loader:
        # Move the data to the GPU
        pssm, label = pssm.to(device), label.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(pssm)
        
        # calculate the loss
        loss = criterion(output, label)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate the accuracy
        train_loss += loss.item()
        
#       # After the training steps, evaluate the model on the validation set
#         valid_loss = 0
#         model.eval()
#         with torch.no_grad():
#             # Inside your validation loop:
#             for pssm, label in val_loader:
#                 pssm, label = pssm.to(device), label.to(device)
#                 output = model(pssm)
#                 # Assuming output is of shape [batch_size, num_classes, sequence_length]
#                 # We permute it to be [batch_size, sequence_length, num_classes] for loss calculation
#                 output = output.permute(0, 2, 1)  # Now output shape is [batch_size, sequence_length, num_classes]
#                 # label is already [batch_size, sequence_length], so no need to change its shape
#                 loss = criterion(output, label)
#                 valid_loss += loss.item()

                
#         valid_loss = valid_loss / len(val_loader)
#         print(f'Epoch {epoch+1}/{epochs}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {valid_loss}')
        
    
    # Print the loss for every10th epoch
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {train_loss / len(train_loader)}')

Epoch 1/51, Training Loss: 0.17241579294204712
Epoch 11/51, Training Loss: 0.07357383604738965
Epoch 21/51, Training Loss: 0.07006081861740383
Epoch 31/51, Training Loss: 0.06787420449226929
Epoch 41/51, Training Loss: 0.06616121917630041
Epoch 51/51, Training Loss: 0.06467438978273109


5. **Optimism**

* ***Install Ax-platfrom***

In [60]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.utils.tutorials.cnn_utils import evaluate, load_mnist, train 


In [61]:
#Using Accelerator: GPU T4 X2 from Kaggle to define a model and put it on the GPU for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ProteinStructurePredictor().to(device)

In [62]:
ax_client = AxClient()
# Design an experiment with required arguments: name, parameters, and objective_name.
ax_client.create_experiment(
    name="Tune_FCN_for_DL",  # The name of the experiment.
    parameters=[
        {
            "name": "lr",  
            "type": "range",  
            "bounds": [1e-6, 0.4],  
            "value_type": "float",  
            "log_scale": True,  
        },
        {
            "name": "momentum",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
    ],
    objectives={"accuracy": ObjectiveProperties(minimize=False)},  # minimization setting.
)

[INFO 03-18 09:31:12] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-18 09:31:12] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter momentum. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-18 09:31:12] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-06, 0.4], log_scale=True), RangeParameter(name='momentum', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 03-18 09:31:12] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there is at least one ordered parameter and there are no unordered categorical parameters.
[INFO 03-18 09:31:12] ax.modelbridge.dispatch_utils: Calcu

In [63]:
def train_evaluate(parameterization):
    dtype = torch.float   
    model = ProteinStructurePredictor()
    model = train(
        net=model,
        train_loader=train_loader,
        parameters=parameterization,
        dtype=dtype,
        device=device,
    )

    return evaluate(
        net=model,
        data_loader=valid_loader,
        dtype=dtype,
        device=device,
    )



In [64]:
ax_client.attach_trial(
    parameters={"lr": 0.000026, "momentum": 0.58}
)

# Get the parameters and run the trial
baseline_parameters = ax_client.get_trial_parameters(trial_index=0)
ax_client.complete_trial(trial_index=0, raw_data=train_evaluate(baseline_parameters))

for i in range(50):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))

best_parameters, values = ax_client.get_best_parameters()
best_parameters

[INFO 03-18 09:31:12] ax.core.experiment: Attached custom parameterizations [{'lr': 2.6e-05, 'momentum': 0.58}] as trial 0.
[INFO 03-18 09:33:22] ax.service.ax_client: Completed trial 0 with data: {'accuracy': (1585.697203, None)}.
[INFO 03-18 09:33:22] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 1.1e-05, 'momentum': 0.74138} using model Sobol.
[INFO 03-18 09:35:30] ax.service.ax_client: Completed trial 1 with data: {'accuracy': (1585.697203, None)}.
[INFO 03-18 09:35:30] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 0.023141, 'momentum': 0.899275} using model Sobol.
[INFO 03-18 09:37:46] ax.service.ax_client: Completed trial 2 with data: {'accuracy': (1585.697203, None)}.
[INFO 03-18 09:37:46] ax.service.ax_client: Generated new trial 3 with parameters {'lr': 0.000981, 'momentum': 0.139496} using model Sobol.
[INFO 03-18 09:40:08] ax.service.ax_client: Completed trial 3 with data: {'accuracy': (1585.697203, None)}.
[INFO 03-18 09:40:08] ax.s

{'lr': 2.6e-05, 'momentum': 0.58}

In [65]:
df = ax_client.get_trials_data_frame()
best_arm_idx = df.trial_index[df["accuracy"] == df["accuracy"].max()].values[0]
best_arm = ax_client.get_trial_parameters(best_arm_idx)
best_arm

[WARNING 03-18 11:32:20] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


{'lr': 2.6e-05, 'momentum': 0.58}

In [66]:
render(ax_client.get_contour_plot(param_x="lr", param_y="momentum", metric_name="accuracy"))

render(
    ax_client.get_optimization_trace()
)

[INFO 03-18 11:32:20] ax.service.ax_client: Retrieving contour plot with parameter 'lr' on X-axis and 'momentum' on Y-axis, for metric 'accuracy'. Remaining parameters are affixed to the middle of their range.


In [67]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss(ignore_index=-100)
# #The optimizer is the Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0005)

lowest_val_loss = float('inf')
best_model = model

# train the model
epochs = 50
# epochs = 100
# epochs = 500
# too long but improve too less
loss_accuracy_list = []

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for pssm, label in train_loader:
        # Move the data to the GPU
        pssm, label = pssm.to(device), label.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(pssm)
        
        # calculate the loss
        loss = criterion(output, label)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate the loss and accuracy
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    
    loss_accuracy_list.append((train_loss / len(train_loader), correct / total))
    # Print the loss for every10th epoch
#     if epoch % 10 == 0:
#         print(f'Epoch {epoch+1}/{epochs}, Training Loss: {train_loss / len(train_loader)}')
        
    # After the training steps, evaluate the model on the validation set
    valid_loss = 0.0
    model.eval()
    with torch.no_grad():
        for pssm, label in valid_loader:
            pssm, label = pssm.to(device), label.to(device)
            output = model(pssm)
            loss = criterion(output, label)
            valid_loss += loss.item()

    valid_loss = valid_loss / len(valid_loader)
    
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {round(train_loss / len(train_loader), 4)}')

        # Save the model if the validation loss has decreased than the previous one
    if epoch == 0:
        lowest_val_loss = valid_loss
        best_model = copy.deepcopy(model.state_dict())
    else:
        if valid_loss < lowest_val_loss:
            lowest_val_loss = valid_loss
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, 'protein_structure_predictor.pt')
            
print('Training is done!')
print(lowest_val_loss)

Epoch 1/50, Training Loss: 0.2118
Epoch 2/50, Training Loss: 0.093
Epoch 3/50, Training Loss: 0.0873
Epoch 4/50, Training Loss: 0.0845
Epoch 5/50, Training Loss: 0.0825
Epoch 6/50, Training Loss: 0.081
Epoch 7/50, Training Loss: 0.0796
Epoch 8/50, Training Loss: 0.0785
Epoch 9/50, Training Loss: 0.0778
Epoch 10/50, Training Loss: 0.0771
Epoch 11/50, Training Loss: 0.0762
Epoch 12/50, Training Loss: 0.0761
Epoch 13/50, Training Loss: 0.0751
Epoch 14/50, Training Loss: 0.0749
Epoch 15/50, Training Loss: 0.0743
Epoch 16/50, Training Loss: 0.0739
Epoch 17/50, Training Loss: 0.0736
Epoch 18/50, Training Loss: 0.0731
Epoch 19/50, Training Loss: 0.0728
Epoch 20/50, Training Loss: 0.0725
Epoch 21/50, Training Loss: 0.0722
Epoch 22/50, Training Loss: 0.0721
Epoch 23/50, Training Loss: 0.0718
Epoch 24/50, Training Loss: 0.0715
Epoch 25/50, Training Loss: 0.0713
Epoch 26/50, Training Loss: 0.0712
Epoch 27/50, Training Loss: 0.0708
Epoch 28/50, Training Loss: 0.0708
Epoch 29/50, Training Loss: 0.0

In [68]:
# now we can load the test data and make predictions
# Apply one_hot_encoding to the test sequences
seqs_test['one_hot_sequence'] = seqs_test['SEQUENCE'].apply(lambda x: encode_amino_acid_sequence(x))

# define a function to load the pssm data from the test folder
def get_pssm_data(filepath):
    pssm_df = pd.read_csv(filepath)
    pssm_df.drop(['RES_NUM', 'AMINO_ACID'], axis=1, inplace=True)
    pssm_np = pssm_df.to_numpy()
    return pssm_np

# Add the PSSM column to the test dataframe
seqs_test['PSSM'] = None
for index, row in seqs_test.iterrows():
    pdb_id = row['PDB_ID']
    
    filepath = os.path.join(pssm_test, f'{pdb_id}_test.csv')
    pssm_np = get_pssm_data(filepath)
    
    seqs_test.at[index, 'PSSM'] = pssm_np
    
# Get the PSSM profiles and pad them to the same length
padded_pssm_test = encode_pssm(seqs_test['PSSM'])
pssm_array = np.stack(padded_pssm_test)
# Convert the numpy arrays to PyTorch tensors
pssm_tensor_test = torch.tensor(pssm_array, dtype=torch.float32)

# Transpose the tensor to the correct shape
pssm_tensor_test = pssm_tensor_test.permute(0, 2, 1)

# Create a DataLoader object for the test set
test_dataset = ProteinDataset(pssm_tensor_test,None)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#Define a function to make predictions for the test set
def make_predictions(model, test_loader):
    predictions = []
    model.eval()
    
    with torch.no_grad():
        for pssm in test_loader:
            pssm = pssm.to(device)
            output = model(pssm)
            
            _, predicted = torch.max(output, 1)
            predictions.extend(predicted.cpu().numpy())
            
    return predictions

# load the best model and make predictions
model.load_state_dict(torch.load('/kaggle/working/protein_structure_predictor.pt'))
predictions = make_predictions(model, test_loader)

# Convert the predictions to the original sequence
for index, row in seqs_test.iterrows():
    seqence=row['SEQUENCE']
    length=len(seqence)
    predictions[index] = predictions[index][:length]

    
# Convert the predictions to the original labels
labels = {0: 'H', 1: 'E', 2: 'C'}
predicted_labels = []
for i in range(len(predictions)):
    predicted_labels.append([labels[p] for p in predictions[i]])


results = []
for index, row in seqs_test.iterrows():
    pdb_id = row['PDB_ID']
    for j, label in enumerate(predicted_labels[index]): 
        results.append({'ID': f'{pdb_id}_{j+1}', 'STRUCTURE': label})


results_df = pd.DataFrame(results)
results_df.to_csv('submission.csv', index=False)


In [69]:
print(results_df.shape)

(43161, 2)
